In [5]:
import pandas as pd

#'CTU-IoT-Malware-Capture-43-1', 'CTU-IoT-Malware-Capture-17-1','CTU-IoT-Malware-Capture-33-1','CTU-IoT-Malware-Capture-39-1',
dataset_name=['CTU-IoT-Malware-Capture-21-1','CTU-IoT-Malware-Capture-42-1','CTU-IoT-Malware-Capture-52-1','CTU-IoT-Malware-Capture-20-1','CTU-IoT-Malware-Capture-60-1','CTU-IoT-Malware-Capture-36-1','CTU-IoT-Malware-Capture-8-1','CTU-IoT-Malware-Capture-35-1','CTU-IoT-Malware-Capture-48-1','CTU-IoT-Malware-Capture-7-1','CTU-IoT-Malware-Capture-9-1','CTU-IoT-Malware-Capture-3-1','CTU-IoT-Malware-Capture-1-1','CTU-IoT-Malware-Capture-34-1','CTU-IoT-Malware-Capture-44-1','CTU-IoT-Malware-Capture-49-1']
dati={}
for data_name in dataset_name:
    dati[data_name] = pd.read_csv(f"/Users/chiara/PycharmProjects/PACKTACK/CleanData/{data_name}.txt", sep="\t", header=0)


In [ ]:
# Standardizzation
from sklearn import preprocessing
import numpy as np


Ys_label={}
Xs_fact, Xs_num, Xs_std={}, {},{}
for dato in dati:
    Ys_label[dato]= dati[dato].label.map({'Benign':0,'Malicious':1})
    Xs_fact = dati[dato].drop(labels=["duration","orig_bytes","resp_bytes","local_orig","local_resp","missed_bytes","tunnel_parents" ,"label","detailed.label"], axis=1)
    Xs_num[dato]= dati[dato].select_dtypes(include=np.number).drop(labels=['ts', 'id.orig_p', 'id.resp_p' ], axis=1)

    Xs_std[dato] = preprocessing.scale(Xs_num[dato])


# Random forest with only numeric features


In [ ]:

# Modelling
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, ConfusionMatrixDisplay
from sklearn.model_selection import RandomizedSearchCV, train_test_split
from scipy.stats import randint

# Tree Visualisation
from sklearn.tree import export_graphviz
from IPython.display import Image

import graphviz

In [ ]:
len(Xs_std[dato])

In [69]:
# Split data
X_train={}
X_test={}
y_train={}
y_test={}
Acc={}
for dato in dati:
    X_train[dato], X_test[dato], y_train[dato], y_test[dato] = train_test_split(Xs_std[dato], Ys_label[dato], test_size=0.2)
    Y=np.array(y_train[dato])
    rf = RandomForestClassifier()
    rf.fit(X_train[dato], y_train[dato])
    y_pred = rf.predict(X_test[dato])
    accuracy = accuracy_score(y_test[dato], y_pred)
    print(f'Accuracy {dato}:', accuracy)

Accuracy CTU-IoT-Malware-Capture-21-1: 1.0
Accuracy CTU-IoT-Malware-Capture-42-1: 1.0
Accuracy CTU-IoT-Malware-Capture-52-1: 0.9999939336876228
Accuracy CTU-IoT-Malware-Capture-20-1: 1.0
Accuracy CTU-IoT-Malware-Capture-60-1: 0.976
Accuracy CTU-IoT-Malware-Capture-36-1: 1.0
Accuracy CTU-IoT-Malware-Capture-8-1: 1.0
Accuracy CTU-IoT-Malware-Capture-35-1: 0.999988362765838
Accuracy CTU-IoT-Malware-Capture-48-1: 1.0
Accuracy CTU-IoT-Malware-Capture-7-1: 0.9998578535891969
Accuracy CTU-IoT-Malware-Capture-9-1: 1.0
Accuracy CTU-IoT-Malware-Capture-3-1: 0.9995131450827653
Accuracy CTU-IoT-Malware-Capture-1-1: 0.9291127324076254
Accuracy CTU-IoT-Malware-Capture-34-1: 0.996244131455399
Accuracy CTU-IoT-Malware-Capture-44-1: 0.96875
Accuracy CTU-IoT-Malware-Capture-49-1: 0.9999990758812396


In [1]:
best_rf={}
for dato in dati:
    param_dist = {'n_estimators': randint(50,500),
                  'max_depth': randint(1,20)}

    # Create a random forest classifier
    rf = RandomForestClassifier()

    # Use random search to find the best hyperparameters
    rand_search = RandomizedSearchCV(rf,
                                     param_distributions = param_dist,
                                     n_iter=5,
                                     cv=5)

    # Fit the random search object to the data
    rand_search.fit( X_train[dato], y_train[dato])
    # Create a variable for the best model
    best_rf[dato] = rand_search.best_estimator_

    # Print the best hyperparameters
    print(f'{dato} - Best hyperparameters:',  rand_search.best_params_)

NameError: name 'dati' is not defined